In [1]:
import pandas as pd
import numpy as np


X_train = pd.read_csv("train_values.csv")
y_train = pd.read_csv("train_labels.csv")

# Join them on building_id
df_train = pd.merge(X_train, y_train, on="building_id")
# Convert the 'damage_grade' to a categorical type

categories = [
    "land_surface_condition",
    "foundation_type",
    "roof_type",
    "ground_floor_type",
    "other_floor_type",
    "position",
    "plan_configuration",
    "legal_ownership_status",
    "damage_grade",
]

for category in categories:
    df_train[category] = df_train[category].astype("category")

df_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,2
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,3
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,2


In [2]:
# Use scikit-learn for feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import joblib


def main(df_train, id_pkl=None, use_all_data=False):
    # Define the features and target
    X = df_train.drop(columns=["building_id", "damage_grade"])
    y = df_train["damage_grade"]

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Define the preprocessing for categorical features
    categorical_features = X_train.select_dtypes(include=["category"]).columns.tolist()
    numerical_features = X_train.select_dtypes(
        include=["float64", "int64"]
    ).columns.tolist()
    # Create a preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", SimpleImputer(strategy="mean"), numerical_features),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ]
    )
    # Create a pipeline with preprocessing and the model
    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(n_estimators=100, random_state=42)),
        ]
    )

    if id_pkl is None:
        loaded = False
    else:
        try:
            pipeline = joblib.load(f"model_{id_pkl}.pkl")
            loaded = True
        except FileNotFoundError:
            loaded = False

    if not loaded:
        # Fit the model
        if use_all_data:
            X_train = X
            y_train = y
        pipeline.fit(X_train, y_train)
        if id_pkl is not None:
            # Save the model
            joblib.dump(pipeline, f"model_{id_pkl}.pkl")

    # Make predictions on the validation set
    y_pred = pipeline.predict(X_val)
    # Evaluate the model
    print(
        classification_report(
            y_val, y_pred, target_names=[str(i) for i in y.cat.categories]
        )
    )

    # Feature importance
    importances = pipeline.named_steps["classifier"].feature_importances_
    # Get feature names after preprocessing
    feature_names = (
        pipeline.named_steps["preprocessor"]
        .transformers_[1][1]
        .get_feature_names_out(categorical_features)
    )
    # Combine feature names with importances
    feature_importances = pd.DataFrame(
        {
            "feature": np.concatenate([numerical_features, feature_names]),
            "importance": importances,
        }
    )
    # Sort by importance
    feature_importances = feature_importances.sort_values(
        by="importance", ascending=False
    )
    print(feature_importances.head(10))
    return pipeline


_ = main(df_train, "as_it_is")

print("Notice that the geo levels are utmost important")

              precision    recall  f1-score   support

           1       0.64      0.48      0.55      5025
           2       0.72      0.82      0.77     29652
           3       0.72      0.60      0.65     17444

    accuracy                           0.72     52121
   macro avg       0.69      0.64      0.66     52121
weighted avg       0.71      0.72      0.71     52121

                                feature  importance
2                        geo_level_3_id    0.152891
1                        geo_level_2_id    0.134381
0                        geo_level_1_id    0.133391
4                                   age    0.121340
5                       area_percentage    0.113307
6                     height_percentage    0.056311
18                       count_families    0.025126
3                   count_floors_pre_eq    0.016402
35                    foundation_type_r    0.014867
8   has_superstructure_mud_mortar_stone    0.013175
Notice that the geo levels are utmost important

In [3]:
set_train = set(df_train["geo_level_3_id"].unique())
df_test = pd.read_csv("test_values.csv")
set_test = set(df_test["geo_level_3_id"].unique())
print(*map(int, set_test - set_train))
print("Not all geo_level_ids in the test set part of the training set")


3584 8193 2 9218 4 5124 7686 1545 2059 10766 5135 4112 1554 2579 5141 3608 12313 4640 6178 10788 9254 4648 4137 1068 5677 558 4143 11310 9265 1075 6709 1595 6204 10812 9790 3135 64 2113 1602 7747 9796 69 7237 3144 7240 11337 4172 5196 3151 12367 595 4691 8790 2655 8800 11872 8803 4196 5733 3686 10851 11876 9833 1131 5227 2669 8301 7791 4725 8822 1143 11896 6265 10874 12409 4220 10367 12415 6785 2182 1671 5256 11400 10378 1675 8844 2701 3725 11402 7315 6295 9368 1694 5282 5285 5798 2727 9383 4780 5804 6319 6831 10931 11959 12473 7355 9924 8389 1734 10953 715 3787 5835 4303 7890 3286 11483 2273 9442 6371 7397 5350 6887 1774 3824 12531 12536 1273 4347 6908 10492 4862 7423 9470 2817 2306 3841 6401 11516 11524 12554 11021 3854 12047 11537 12566 5401 4379 3357 799 7968 11041 6435 1830 7979 9004 9520 1330 9522 1335 3384 1337 10040 2877 1855 6975 833 6978 2371 9027 9537 6984 329 6985 5963 3404 11592 8528 5970 339 8533 9558 11095 10584 2396 1373 3933 6495 5473 5474 1891 3429 2406 871 1395 6516 

In [4]:
arr_levels = locals().get('arr_levels', None)
if arr_levels is None:
    values = {
        i: np.zeros((1 + max(df_train[s]), 4))
        for i, s in enumerate(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"])
    }
    arr_levels = []
    for i, row in df_train.iterrows():
        levels = [row["geo_level_1_id"], row["geo_level_2_id"], row["geo_level_3_id"]]
        arr_levels.append(levels)
        y = row["damage_grade"]
        for j, level in enumerate(levels):
            values[j][level, y] += 1

    arr_levels = np.array(arr_levels)


def get_geo_y(df, num=50, drop_original=False):
    # Get the average y for each geo_level_3, using larger levels if n_samples < num
    arr_levels = df[["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"]].values
    out = []
    out_by_level3 = {}
    for lvl in arr_levels:
        if lvl[2] in out_by_level3:
            # Already computed for this level 3, skip
            out.append(out_by_level3[lvl[2]])
            continue

        # Values by level (and y)
        vals1 = values[0][lvl[0]]
        vals2 = values[1][lvl[1]]
        vals3 = values[2][lvl[2]]

        if vals3.sum() >= num:  # Enough samples at local level
            loc = vals3
            w = 1.0
            sup = vals3 * 0
        elif vals2.sum() >= num:  # Not enough at local level, but enough at next level
            loc = vals3
            w = vals3.sum() / vals2.sum()
            sup = vals2 - vals3
        else:  # Not enough at local and next level, use the next level up
            loc = vals2
            w = vals2.sum() / vals1.sum()
            sup = vals1 - vals2

        # Compute the statistic, greedily weighing the local level
        loc = loc / (1e-9 + loc.sum())
        sup = sup / (1e-9 + sup.sum())
        tup = w * loc + (1 - w) * sup
        tup = tup[1:]
        out.append(tup)
        out_by_level3[lvl[2]] = tup

    out = np.array(out)
    out = {
        "geo_y1": out[:, 0],
        "geo_y2": out[:, 1],
        "geo_y3": out[:, 2],
    }
    out = pd.DataFrame(out)
    if drop_original:
        df = df.drop(columns=["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"])
        out = pd.concat([df, out], axis=1)
    return out


In [5]:
print("How to choose the minimum number of samples?")
print(df_train.groupby("geo_level_1_id")["geo_level_1_id"].apply(len).describe())
print(df_train.groupby("geo_level_2_id")["geo_level_2_id"].apply(len).describe())
print(df_train.groupby("geo_level_3_id")["geo_level_3_id"].apply(len).describe())

How to choose the minimum number of samples?
count       31.000000
mean      8406.483871
std       7922.972472
min        265.000000
25%       2503.000000
50%       4332.000000
75%      14728.500000
max      24381.000000
Name: geo_level_1_id, dtype: float64
count    1414.000000
mean      184.300566
std       259.928137
min         1.000000
25%        27.000000
50%       120.500000
75%       256.000000
max      4038.000000
Name: geo_level_2_id, dtype: float64
count    11595.000000
mean        22.475291
std         29.507407
min          1.000000
25%          5.000000
50%         14.000000
75%         30.000000
max        651.000000
Name: geo_level_3_id, dtype: float64


In [11]:
for num in [2, 3, 5, 25, 50]:
    print(f"Using at least num={num} samples to aggregate geo level")
    df_pre = get_geo_y(df_train, num=num, drop_original=True)
    main(df_pre, f"num_{num}")
    print()


Using at least num=2 samples to aggregate geo level
              precision    recall  f1-score   support

           1       0.66      0.57      0.61      5025
           2       0.76      0.82      0.79     29652
           3       0.74      0.67      0.70     17444

    accuracy                           0.75     52121
   macro avg       0.72      0.69      0.70     52121
weighted avg       0.74      0.75      0.74     52121

                                feature  importance
29                               geo_y3    0.230576
28                               geo_y2    0.193886
1                                   age    0.101890
2                       area_percentage    0.091794
27                               geo_y1    0.089905
3                     height_percentage    0.046611
15                       count_families    0.020598
0                   count_floors_pre_eq    0.014517
35                    foundation_type_r    0.012293
5   has_superstructure_mud_mortar_stone    0.01

In [12]:
# Predict on the test set

num = 3

df_alt = get_geo_y(df_train, num=num, drop_original=True)
pipeline = main(df_alt, f"num_{num}_final", use_all_data=True)

df_test = pd.read_csv("test_values.csv")
df_test_pre = get_geo_y(df_test, num=num, drop_original=True)

y_test_pred = pipeline.predict(df_test_pre)
df_test["damage_grade"] = y_test_pred

df_submission = df_test[["building_id", "damage_grade"]]
df_submission.to_csv(f"submission_num_{num}.csv", index=False)
print(f"Submission saved to submission_num_{num}.csv")


              precision    recall  f1-score   support

           1       0.99      0.98      0.98      5025
           2       0.98      0.99      0.98     29652
           3       0.98      0.97      0.98     17444

    accuracy                           0.98     52121
   macro avg       0.98      0.98      0.98     52121
weighted avg       0.98      0.98      0.98     52121

                                feature  importance
29                               geo_y3    0.227082
28                               geo_y2    0.194802
1                                   age    0.104486
2                       area_percentage    0.094442
27                               geo_y1    0.087810
3                     height_percentage    0.046714
15                       count_families    0.021189
0                   count_floors_pre_eq    0.014245
35                    foundation_type_r    0.012600
5   has_superstructure_mud_mortar_stone    0.011668
Submission saved to submission_num_3.csv
